# Milestone 1 - CasierVert952

In [12]:
# Import the requiered libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
with open('data/ratings_ba.txt', 'r') as fichier:
    for i, ligne in enumerate(fichier):
        if i >= 100000:  # Arrête la boucle après 100 lignes
            break
        print(ligne.strip())

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.8
date: 1417431600
user_name: Hellpop65
user_id: hellpop65.48993
appearance: nan
aroma: nan
palate: nan
taste: nan
overall: nan
rating: 3.25
text: nan
review: False

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.8
date: 1401357600
user_name: Latarnik
user_id: latarnik.52897
appearance: nan
aroma: nan
palate: nan
taste: nan
overall: nan
rating: 3.50
text: nan
review: False

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.8
date: 1393412400
user_name: RochefortChris
user_id: rochefortchris.697017
appearance: nan
aroma: nan
palate: nan
taste: nan
overall: nan
rating: 3.50
text: nan
review: False

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
br

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 7437: character maps to <undefined>

In [11]:
with open('data/reviews.txt', 'r') as fichier:
    for i, ligne in enumerate(fichier):
        if i >= 100000:  # Arrête la boucle après 100 lignes
            break
        print(ligne.strip())

beer_name: RÃ©gab
beer_id: 142544
brewery_name: Societe des Brasseries du Gabon (SOBRAGA)
brewery_id: 37262
style: Euro Pale Lager
abv: 4.5
date: 1440064800
user_name: nmann08
user_id: nmann08.184925
appearance: 3.25
aroma: 2.75
palate: 3.25
taste: 2.75
overall: 3.0
rating: 2.88
text: From a bottle, pours a piss yellow color with a fizzy white head.  This is carbonated similar to soda.The nose is basic.. malt, corn, a little floral, some earthy straw.  The flavor is boring, not offensive, just boring.  Tastes a little like corn and grain.  Hard to write a review on something so simple.Its ok, could be way worse.

beer_name: Barelegs Brew
beer_id: 19590
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.5
date: 1235127600
user_name: StJamesGate
user_id: stjamesgate.163714
appearance: 3.0
aroma: 3.5
palate: 3.5
taste: 4.0
overall: 3.5
rating: 3.67
text: Pours pale copper with a thin head that quickly goes. Caramel, golden syrup nose. Taste

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 5003: character maps to <undefined>